In [1]:
import urllib
from urllib import request
import requests
import csv

import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from xml.etree import cElementTree as ET 
from bs4 import BeautifulSoup

In [2]:
# 보건복지부_코로나19 확진자 성별 연령별 현황

sv_key ="ENdJ67X0/9dtrA0/F3/coohsMZEtnO2vmKZm2G0uxJPcpqO8mhTQii+vnwJOxGiPjFgnzfZD49HT4ZR97uyGLw=="
url = 'http://apis.data.go.kr/1352000/ODMS_COVID_05/callCovid05Api'
params ={'serviceKey' : sv_key, 'numOfRows' : '100', 'apiType' : 'JSON'}

page_no = 1 
page_list = []
while True:
    print(page_no)
    params ['pageNo'] = page_no
    response = requests.get(url, params=params)
    rescode = response.status_code # 응답 코드 읽기

    # print(rescode)
    # print(response.content)

    if(rescode==200): # 200 : 정상응답
        temp_dict = response.json() # json 결과를 읽어서 dict로 변환
        page_list= page_list + temp_dict['items']
        if len(page_list) == temp_dict['totalCount']:
            break
        page_no=page_no +1 
    else:
        print("Error Code:" + str(rescode))
        break


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150


In [3]:
print(f"전체 데이터 개수: {len(page_list)}")

전체 데이터 개수: 14916


In [4]:
covid_age_gender=pd.DataFrame(page_list)
covid_age_gender

,criticalRate,death,deathRate,confCaseRate,createDt,confCase,gubun
0,5.26,600,28.24,5.37,2021-08-09,11409,70-79
1,5.41,591,28.37,5.95,2021-07-28,10922,70-79
2,2.94,1,100.00,41.46,2020-02-20,34,남성
3,0.07,15,0.78,15.00,2021-05-24,20474,40-49
4,1.68,275,52.28,47.72,2020-11-30,16321,남성
...,...,...,...,...,...,...,...
14911,0.1,17741,50.7,53.84,2023-06-22,17270841,여성
14912,0,165,0.46,14.69,2023-08-15,4987756,30-39
14913,0.00,0,0.00,5.49,2020-07-29,782,10-19
14914,0.00,0,0.00,1.71,2020-07-27,243,0-9


In [13]:
covid_age_gender.replace('','0',inplace=True)

In [14]:
print("각 컬럼별 0 값 개수")
print((covid_age_gender == '0').sum())

print("\n각 컬럼별 NaN 값 개수")
print(covid_age_gender.isna().sum())
print("\n각 컬럼별 '' 값 개수")
print((covid_age_gender == '').sum())

각 컬럼별 0 값 개수
치명률      3193
사망자수     2517
사망률      1153
확진률         0
등록일자        0
확진자수      151
연령,나이       0
dtype: int64

각 컬럼별 NaN 값 개수
치명률      0
사망자수     0
사망률      0
확진률      0
등록일자     0
확진자수     0
연령,나이    0
dtype: int64

각 컬럼별 '' 값 개수
치명률      0
사망자수     0
사망률      0
확진률      0
등록일자     0
확진자수     0
연령,나이    0
dtype: int64


In [6]:
new_column_names = {
    'criticalRate': '치명률',
    'death': '사망자수',
    'deathRate': '사망률',
    'confCaseRate': '확진률',
    'createDt': '등록일자',
    'confCase': '확진자수',
    'gubun': '연령,나이'
    
}

# 컬럼 이름 변경
covid_age_gender.rename(columns=new_column_names, inplace=True)

# 결과 출력
print(covid_age_gender)

        치명률   사망자수     사망률    확진률        등록일자      확진자수  연령,나이
0      5.26    600   28.24   5.37  2021-08-09     11409  70-79
1      5.41    591   28.37   5.95  2021-07-28     10922  70-79
2      2.94      1  100.00  41.46  2020-02-20        34     남성
3      0.07     15    0.78  15.00  2021-05-24     20474  40-49
4      1.68    275   52.28  47.72  2020-11-30     16321     남성
...     ...    ...     ...    ...         ...       ...    ...
14911   0.1  17741    50.7  53.84  2023-06-22  17270841     여성
14912     0    165    0.46  14.69  2023-08-15   4987756  30-39
14913  0.00      0    0.00   5.49  2020-07-29       782  10-19
14914  0.00      0    0.00   1.71  2020-07-27       243    0-9
14915  0.06     25    1.05  16.01  2021-09-15     44497  40-49

[14916 rows x 7 columns]


In [20]:
covid_age_gender['등록일자'] = pd.to_datetime(covid_age_gender['등록일자'])
covid_age_gender['사망자수'] = covid_age_gender['사망자수'].astype(int)
#covid_age_gender['사망률'] = covid_age_gender['사망률'].astype(int)
covid_age_gender['사망률'] = covid_age_gender['사망률'].apply(lambda x: round(float(x)) if x != '' else 0)
covid_age_gender['확진률'] = covid_age_gender['확진률'].apply(lambda x: round(float(x)) if x != '' else 0)
#covid_age_gender['확진률'] = covid_age_gender['확진률'].astype(int)

#covid_age_gender['치명률'] = covid_age_gender['치명률'].astype(int)
covid_age_gender['확진자수'] = covid_age_gender['확진자수'].astype(int)
#covid_age_gender['연령,나이'] = covid_age_gender['연령,나이'].astype(int)
covid_age_gender.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14916 entries, 0 to 14915
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   치명률     14916 non-null  object        
 1   사망자수    14916 non-null  int64         
 2   사망률     14916 non-null  int64         
 3   확진률     14916 non-null  int64         
 4   등록일자    14916 non-null  datetime64[ns]
 5   확진자수    14916 non-null  int64         
 6   연령,나이   14916 non-null  object        
dtypes: datetime64[ns](1), int64(4), object(2)
memory usage: 815.8+ KB


In [7]:
covid_age_gender

,치명률,사망자수,사망률,확진률,등록일자,확진자수,"연령,나이"
0,5.26,600,28.24,5.37,2021-08-09,11409,70-79
1,5.41,591,28.37,5.95,2021-07-28,10922,70-79
2,2.94,1,100.00,41.46,2020-02-20,34,남성
3,0.07,15,0.78,15.00,2021-05-24,20474,40-49
4,1.68,275,52.28,47.72,2020-11-30,16321,남성
...,...,...,...,...,...,...,...
14911,0.1,17741,50.7,53.84,2023-06-22,17270841,여성
14912,0,165,0.46,14.69,2023-08-15,4987756,30-39
14913,0.00,0,0.00,5.49,2020-07-29,782,10-19
14914,0.00,0,0.00,1.71,2020-07-27,243,0-9


In [ ]:
# 전연령대에서 남성의 사망자수 합계
male_deaths_all_ages = covid_age_gender[covid_age_gender['연령,나이'] == '남성']['사망자수'].sum()

# 전연령대에서 여성의 사망자수 합계
female_deaths_all_ages = covid_age_gender[covid_age_gender['연령,나이'] == '여성']['사망자수'].sum()

# 결과 출력
print(f" 남성의 사망자수 합계: {male_deaths_all_ages}")
print(f" 여성의 사망자수 합계: {female_deaths_all_ages}")

전연령대 남성의 사망자수 합계: 8321884
전연령대 여성의 사망자수 합계: 8603737


In [ ]:
# 전연령대에서 남성의 사망률 합계
male_deathrate = covid_age_gender[covid_age_gender['연령,나이'] == '남성']['사망률'].mean()

# 전연령대에서 여성의 사망률 합계
female_deathrate = covid_age_gender[covid_age_gender['연령,나이'] == '여성']['사망률'].mean()

# 결과 출력
print(f" 남성의 사망률 : {male_deathrate}")
print(f" 여성의 사망률 : {female_deathrate}")

전연령대 남성의 사망률 : 58.08259587020649
전연령대 여성의 사망률 : 57.39601769911504


In [ ]:
# 전연령대에서 남성의 확진률 합계
male_conrate = covid_age_gender[covid_age_gender['연령,나이'] == '남성']['확진률'].mean()

# 전연령대에서 여성의 확진률 합계
female_conrate = covid_age_gender[covid_age_gender['연령,나이'] == '여성']['확진률'].mean()

# 결과 출력
print(f" 남성의 확진률 : {male_conrate}")
print(f" 여성의 확진률 : {female_conrate}")

전연령대 남성의 확진률 : 47.40339233038348
전연령대 여성의 확진률 : 52.59660766961652
